<a href="https://colab.research.google.com/github/jmestanza/natural-language-processing-practice/blob/main/desafios/Desafio_3/Desafio_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Desafío 3
Entrenar un modelo de lenguaje basado en caracteres utilizando la [notebook](https://github.com/jmestanza/procesamiento_lenguaje_natural/blob/main/clase_3/ejercicios/3_modelo_lenguaje_char.ipynb) de clase como base .

Objetivos principales
- Comprender el modelo de lenguaje basado en caracteres.
- Definir un corpus para entrenar el modelo.
- Tomar decisiones sobre la arquitectura del modelo.
- Ejecutar el entrenamiento y analizar los resultados.
- Generar secuencias de texto utilizando el modelo entrenado.

Descripción de la tarea
Se proporciona un notebook base que trabaja con el libro La vuelta al mundo en 80 días.

Se realiza preprocesamiento del texto, eliminando etiquetas HTML y convirtiendo todo a minúsculas para reducir el vocabulario.

Se define un vocabulario de caracteres, incluyendo letras, signos de puntuación y espacios.

Se crean diccionarios de tokenización (carácter a índice y viceversa).

Se divide el texto en conjuntos de entrenamiento y validación, tomando el 10% del texto para validación.

El modelo se entrena en un esquema many-to-many, donde la entrada es una secuencia de caracteres y la salida es la misma secuencia desplazada en un carácter.

Se menciona el uso de capa embedding y time-distributed layers para manejar secuencias.

Se mide la perplejidad del modelo como métrica de evaluación.

Generación de texto
Se propone experimentar con diferentes estrategias de generación de secuencias.

Se pueden probar enfoques deterministas o estocásticos para generar texto.

Se debe documentar los resultados de generación obtenidos con el modelo final.

Entrega esperada
Modelo de lenguaje entrenado con la arquitectura seleccionada.

Ejemplos de generación de texto con el modelo.

Reflexión sobre los resultados y ajustes realizados.

En resumen, el trabajo implica entrenar un modelo de lenguaje basado en caracteres, ajustar su arquitectura, medir su rendimiento y generar texto con él.

## Dataset
Originalmente, se había optado por el dataset `THE COMPLETE SHERLOCK HOLMES` que tiene todo el canon de los libros de Arthur Conan Doyle. Pero debido a que es un corpus muy grande  (3868122 de caracteres), la sesión se queda sin ram. Por lo cual se optó por un corpus más chico `The Adventures of Sherlock Holmes` (581565 caracteres).

In [9]:
import os
dataset_path = 'the_adventures_of_sherlock_holmes.txt'
if not os.path.exists(dataset_path):
  # !wget https://raw.githubusercontent.com/jmestanza/natural-language-processing-practice/refs/heads/main/desafios/Desafio_2/cano.txt
  !wget https://raw.githubusercontent.com/jmestanza/natural-language-processing-practice/refs/heads/main/desafios/Desafio_2/the_adventures_of_sherlock_holmes.txt

In [10]:
with open(dataset_path, 'r') as f:
  text = f.read()

print(len(text))
print(text[:1000])

581565
﻿The Project Gutenberg eBook of The Adventures of Sherlock Holmes
    
This ebook is for the use of anyone anywhere in the United States and
most other parts of the world at no cost and with almost no restrictions
whatsoever. You may copy it, give it away or re-use it under the terms
of the Project Gutenberg License included with this ebook or online
at www.gutenberg.org. If you are not located in the United States,
you will have to check the laws of the country where you are located
before using this eBook.

Title: The Adventures of Sherlock Holmes

Author: Arthur Conan Doyle

Release date: March 1, 1999 [eBook #1661]
                Most recently updated: October 10, 2023

Language: English

Credits: an anonymous Project Gutenberg volunteer and Jose Menendez


*** START OF THE PROJECT GUTENBERG EBOOK THE ADVENTURES OF SHERLOCK HOLMES ***




The Adventures of Sherlock Holmes

by Arthur Conan Doyle


Contents

   I.     A Scandal in Bohemia
   II.    The Red-Headed League
   II

## Elegir el tamaño del contexto

En este caso, como el modelo de lenguaje es por caracteres, todo un gran corpus de texto puede ser considerado un documento en sí mismo y el tamaño de contexto puede ser elegido con más libertad en comparación a un modelo de lenguaje tokenizado por palabras y dividido en documentos más acotados.

In [11]:
# seleccionamos el tamaño de contexto
max_context_size = 100

# Usaremos las utilidades de procesamiento de textos y secuencias de Keras
from tensorflow.keras.utils import pad_sequences # se utilizará para padding


In [12]:
# en este caso el vocabulario es el conjunto único de caracteres que existe en todo el texto
chars_vocab = set(text)

# la longitud de vocabulario de caracteres es:
len(chars_vocab)

98

In [13]:
# Construimos los dicionarios que asignan índices a caracteres y viceversa.
# El diccionario `char2idx` servirá como tokenizador.
char2idx = {k: v for v,k in enumerate(chars_vocab)}
idx2char = {v: k for k,v in char2idx.items()}

## Tokenizar

In [14]:
# tokenizamos el texto completo
tokenized_text = [char2idx[ch] for ch in text]
tokenized_text[:25]


[68,
 91,
 59,
 28,
 70,
 41,
 18,
 5,
 36,
 28,
 25,
 80,
 70,
 65,
 66,
 80,
 28,
 6,
 54,
 28,
 18,
 34,
 70,
 28,
 53]

## Organizando y estructurando el dataset

In [15]:
import numpy as np
# separaremos el dataset entre entrenamiento y validación.
# `p_val` será la proporción del corpus que se reservará para validación
# `num_val` es la cantidad de secuencias de tamaño `max_context_size` que se usará en validación
p_val = 0.1
num_val = int(np.ceil(len(tokenized_text)*p_val/max_context_size))

In [16]:
# separamos la porción de texto utilizada en entrenamiento de la de validación.
train_text = tokenized_text[:-num_val*max_context_size]
val_text = tokenized_text[-num_val*max_context_size:]

In [17]:
tokenized_sentences_train = [train_text[init:init+max_context_size] for init in range(len(train_text)-max_context_size+1)]
tokenized_sentences_val = [val_text[init*max_context_size:init*(max_context_size+1)] for init in range(num_val)]

In [18]:
X = np.array(tokenized_sentences_train[:-1])
y = np.array(tokenized_sentences_train[1:])

Nótese que estamos estructurando el problema de aprendizaje como many-to-many:

Entrada: secuencia de tokens [
x0,x1
, ..., xn
]

Target: secuencia de tokens [
  x1,
  x2,
,
, ...,
xn+1
]

De manera que la red tiene que aprender que su salida deben ser los tokens desplazados en una posición y un nuevo token predicho (el N+1).

La ventaja de estructurar el aprendizaje de esta manera es que para cada token de target se propaga una señal de gradiente por el grafo de cómputo recurrente, que es mejor que estructurar el problema como many-to-one en donde sólo una señal de gradiente se propaga.

En este punto tenemos en la variable tokenized_sentences los versos tokenizados. Vamos a quedarnos con un conjunto de validación que utilizaremos para medir la calidad de la generación de secuencias con la métrica de Perplejidad.

In [19]:
X.shape

(523265, 100)

In [20]:
X[0,:10]

array([68, 91, 59, 28, 70, 41, 18,  5, 36, 28])

In [21]:
y[0,:10]

array([91, 59, 28, 70, 41, 18,  5, 36, 28, 25])

In [23]:
vocab_size = len(chars_vocab)
vocab_size

98

## Definiendo el modelo

In [24]:
from keras.layers import Input, TimeDistributed, CategoryEncoding, SimpleRNN, Dense
from keras.models import Model, Sequential

El modelo que se propone como ejemplo consume los índices de los tokens y los transforma en vectores OHE (en este caso no entrenamos una capa de embedding para caracteres). Esa transformación se logra combinando las capas CategoryEncoding que transforma a índices a vectores OHE y TimeDistributed que aplica la capa a lo largo de la dimensión "temporal" de la secuencia.

In [25]:
model = Sequential()

model.add(TimeDistributed(CategoryEncoding(num_tokens=vocab_size, output_mode = "one_hot"),input_shape=(None,1)))
model.add(SimpleRNN(200, return_sequences=True, dropout=0.1, recurrent_dropout=0.1 ))
model.add(Dense(vocab_size, activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy', optimizer='rmsprop')

model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/wrapper.py:27: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ time_distributed (TimeDistributed)   │ (None, None, 98)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ simple_rnn (SimpleRNN)               │ (None, None, 200)           │          59,800 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, None, 98)            │          19,698 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 79,498 (310.54 KB)

 Trainable params: 79,498 (310.54 KB)

 Non-trainable params: 0 (0.00 B)

## Perplexity callback

In [26]:
from tensorflow import keras
class PplCallback(keras.callbacks.Callback):

    '''
    Este callback es una solución ad-hoc para calcular al final de cada epoch de
    entrenamiento la métrica de Perplejidad sobre un conjunto de datos de validación.
    La perplejidad es una métrica cuantitativa para evaluar la calidad de la generación de secuencias.
    Además implementa la finalización del entrenamiento (Early Stopping)
    si la perplejidad no mejora después de `patience` epochs.
    '''

    def __init__(self, val_data, history_ppl,patience=5):
      # El callback lo inicializamos con secuencias de validación sobre las cuales
      # mediremos la perplejidad
      self.val_data = val_data

      self.target = []
      self.padded = []

      count = 0
      self.info = []
      self.min_score = np.inf
      self.patience_counter = 0
      self.patience = patience

      # nos movemos en todas las secuencias de los datos de validación
      for seq in self.val_data:

        len_seq = len(seq)
        # armamos todas las subsecuencias
        subseq = [seq[:i] for i in range(1,len_seq)]
        self.target.extend([seq[i] for i in range(1,len_seq)])

        if len(subseq)!=0:

          self.padded.append(pad_sequences(subseq, maxlen=max_context_size, padding='pre'))

          self.info.append((count,count+len_seq))
          count += len_seq

      self.padded = np.vstack(self.padded)


    def on_epoch_end(self, epoch, logs=None):

        # en `scores` iremos guardando la perplejidad de cada secuencia
        scores = []

        predictions = self.model.predict(self.padded,verbose=0)

        # para cada secuencia de validación
        for start,end in self.info:

          # en `probs` iremos guardando las probabilidades de los términos target
          probs = [predictions[idx_seq,-1,idx_vocab] for idx_seq, idx_vocab in zip(range(start,end),self.target[start:end])]

          # calculamos la perplejidad por medio de logaritmos
          scores.append(np.exp(-np.sum(np.log(probs))/(end-start)))

        # promediamos todos los scores e imprimimos el valor promedio
        current_score = np.mean(scores)
        history_ppl.append(current_score)
        print(f'\n mean perplexity: {current_score} \n')

        # chequeamos si tenemos que detener el entrenamiento
        if current_score < self.min_score:
          self.min_score = current_score
          self.model.save("my_model.keras")
          print("Saved new model!")
          self.patience_counter = 0
        else:
          self.patience_counter += 1
          if self.patience_counter == self.patience:
            print("Stopping training...")
            self.model.stop_training = True

## Entrenamiento

In [ ]:
# fiteamos, nótese el agregado del callback con su inicialización. El batch_size lo podemos seleccionar a mano
# en general, lo mejor es escoger el batch más grande posible que minimice el tiempo de cada época.
# En la variable `history_ppl` se guardarán los valores de perplejidad para cada época.
history_ppl = []
hist = model.fit(X, y, epochs=20, callbacks=[PplCallback(tokenized_sentences_val,history_ppl)], batch_size=256)


Epoch 1/20
2045/2045 ━━━━━━━━━━━━━━━━━━━━ 0s 462ms/step - loss: 2.4374